In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [ ]:
visited = set()

def crawl(url, depth=3):
    print("crawling ", url)
    if url in visited or depth == -1:
        visited.add(url)
        return
    visited.add(url)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        for link in soup.find_all("a", href=True):
            next_url = urljoin(url, link["href"])
            crawl(next_url, depth - 1)
    except:
        pass

start_url = "https://virginia.edu/academics"
crawl(start_url)
print("\n".join(visited))


crawling  https://virginia.edu/academics
crawling  https://virginia.edu/academics#main-content
crawling  https://virginia.edu/academics#main-content
crawling  https://virginia.edu/
crawling  https://virginia.edu/#main-content
crawling  https://virginia.edu/#main-content
crawling  https://virginia.edu/
crawling  https://sisuva.admin.virginia.edu/ihprd/signon.html
crawling  https://canvas.virginia.edu
crawling  http://its.virginia.edu/switchboard/
crawling  https://hr.virginia.edu/careers-uva/job-openings
crawling  https://virginia.edu/alumni
crawling  https://students.virginia.edu/
crawling  https://virginia.edu/facultystaff
crawling  https://virginia.edu/parents
crawling  https://virginia.edu/#search-close
crawling  https://virginia.edu/aboutuva
crawling  https://virginia.edu/aboutuva
crawling  https://virginia.edu/life/charlottesville
crawling  https://virginia.edu/facts
crawling  https://federalinfo.virginia.edu/
crawling  https://president.virginia.edu/
crawling  https://strategicpl

In [11]:
visited = {
    entry for entry in visited
    if " " not in entry and not entry.startswith(("mailto", "tel"))
}

visited

{'http://accessibility.virginia.edu/',
 'http://americanstudies.as.virginia.edu/major-overview',
 'http://archaeology.virginia.edu/',
 'http://art.as.virginia.edu/art-history',
 'http://art.as.virginia.edu/majors-minors',
 'http://as.virginia.edu/',
 'http://asl.virginia.edu/asl-program',
 'http://astronomy.as.virginia.edu/majoring-astronomy',
 'http://batten.virginia.edu/',
 'http://bio.as.virginia.edu/undergraduate/degrees/how-to-declare',
 'http://chemistry.as.virginia.edu/chemistry-major',
 'http://classics.as.virginia.edu/major-and-minor-classics',
 'http://college.as.virginia.edu/LASE_ELA',
 'http://college.as.virginia.edu/awards/imp',
 'http://college.as.virginia.edu/awardsandhonors',
 'http://college.as.virginia.edu/css',
 'http://eastasian.as.virginia.edu/majors-minors',
 'http://education.virginia.edu/academics/programs/global-studies-education-undergraduate-minor',
 'http://educationabroad.virginia.edu/',
 'http://english.as.virginia.edu/english-major',
 'http://eocr.virgini